# Ch5

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp

sns.set()
%matplotlib inline

/Users/hiroaki/.pyenv/versions/3.6.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/hiroaki/.pyenv/versions/3.6.0/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## カテゴライズ変数のエンコーディング
- one-hot:全ての特徴量の中で、一つだけ1となり、ほかは0
- dummy-coding: one-hotの次元から不要な一次元を取り除く
- effect-coding: TODO

In [4]:
# onehotとdummy-encodeで都市空間から家賃を予測
from sklearn  import linear_model

df_rent=pd.DataFrame(
    {'City': ['SF', 'SF', 'SF', 'NYC', 'NYC', 'NYC', 
        'Seattle', 'Seattle', 'Seattle'],
        'Rent': [3999, 4000, 4001, 3499, 3500, 3501, 2499, 2500, 2501]})

df_rent

,City,Rent
0,SF,3999
1,SF,4000
2,SF,4001
3,NYC,3499
4,NYC,3500
5,NYC,3501
6,Seattle,2499
7,Seattle,2500
8,Seattle,2501


In [8]:
one_hot_city=pd.get_dummies(df_rent,prefix='city')
one_hot_city

,Rent,city_NYC,city_SF,city_Seattle
0,3999,0,1,0
1,4000,0,1,0
2,4001,0,1,0
3,3499,1,0,0
4,3500,1,0,0
5,3501,1,0,0
6,2499,0,0,1
7,2500,0,0,1
8,2501,0,0,1


In [56]:
dummy_city=pd.get_dummies(df_rent,prefix='city',drop_first=True)

dummy_city

,Rent,city_SF,city_Seattle
0,3999,1,0
1,4000,1,0
2,4001,1,0
3,3499,0,0
4,3500,0,0
5,3501,0,0
6,2499,0,1
7,2500,0,1
8,2501,0,1


In [55]:
# 線形モデルで禁じ
model=linear_model.LinearRegression()
model.fit(one_hot_city[['city_NYC','city_SF','city_Seattle']]
                   ,one_hot_city['Rent'])


print(model.coef_+model.intercept_)
print(df_rent.groupby('City').mean())

[3500. 4000. 2500.]
         Rent
City         
NYC      3500
SF       4000
Seattle  2500


- one-hotの場合、それぞれの係数にバイアスを足したものが平均となる

In [61]:
model.fit(dummy_city[['city_SF','city_Seattle']]
                  , dummy_city['Rent'])


print(model.coef_+model.intercept_)
print(df_rent.groupby('City').mean())

[4000. 2500.]
         Rent
City         
NYC      3500
SF       4000
Seattle  2500


- dummy_codingの場合、biasはdropした特徴量の平均となる

## カテゴリ数が非常に大きな場合、どのようにすればよいか

### Feature Hashing